# 광고영상 프레임들 훈련용 / 검증용 셋으로 편입 

In [29]:
import os 
import sys
import shutil 

sys.path.append('/Users/kibeomkim/Desktop/영상프레임/')

video_list = os.listdir('/Users/kibeomkim/Desktop/영상프레임')
video_list.remove('.DS_Store')
# 훈련용 비디오 리스트 
train_video_list = video_list[:25]
# 테스트용 비디오 리스트 
test_video_list = video_list[25:]

for video in train_video_list : 
    for frame in os.listdir(f'/Users/kibeomkim/Desktop/영상프레임/{video}') : # 훈련용 비디오 프레임 하나하나에 대해서 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/영상프레임/{video}/{frame}', f'/Users/kibeomkim/Desktop/openlogo_video/train/samsung/{frame}')

for video in test_video_list : 
    for frame in os.listdir(f'/Users/kibeomkim/Desktop/영상프레임/{video}') : # 테스트 비디오 프레임 하나하나에 대해서 
        shutil.copyfile(f'/Users/kibeomkim/Desktop/영상프레임/{video}/{frame}', f'/Users/kibeomkim/Desktop/openlogo_video/test/samsung/{frame}')

# Xception 모델에 갤럭시 S21 광고 영상 프레임들 넣기 

In [31]:
import keras 
from keras.applications import xception
from keras import layers
from keras import preprocessing

base_model = keras.applications.xception.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))

norm_layer = keras.layers.BatchNormalization()
x = norm_layer(inputs)

x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)  # Regularize with dropout
outputs = keras.layers.Dense(2)(x)
model = keras.Model(inputs, outputs)

model.summary()

2022-02-22 23:26:16.921471: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-22 23:26:16.922018: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 batch_normalization_4 (Batc  (None, 150, 150, 3)      12        
 hNormalization)                                                 
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                   

In [33]:
# 이미지 데이터 전처리 # 데이터 증식 사용
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.6,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 검증데이터는 증식하지 않는다.

train_generator = train_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_video/train', 
    target_size=(150, 150),
    batch_size=100, 
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/Users/kibeomkim/Desktop/openlogo_video/test',
    target_size=(150,150),
    batch_size=100,
    class_mode='binary'
)

Found 40783 images belonging to 2 classes.
Found 17106 images belonging to 2 classes.


In [34]:
# 가장 기본적인 3가지 콜백 
callbacks_list = [

    #과대적합 발생 시 조기 스톱 콜백
    keras.callbacks.EarlyStopping(
        monitor='binary_accuracy', # 모델 검증 손실 모니터링
        patience= 10), # 20에폭동안 과대적합 관찰
    
    # 최고의 가중치 자동 저장 콜백
    keras.callbacks.ModelCheckpoint(
        '/Users/kibeomkim/Desktop/models_saved/my_model.h5', 
        monitor = 'val_loss', 
        save_best_only = True), 

    # 학습률 자동 조정 콜백
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=5
    ),

    # 텐서보드 
    keras.callbacks.TensorBoard(
        log_dir = '/Users/kibeomkim/Desktop/my_log_dir',
        histogram_freq = 1, # 1에포크마다 층 출력의 히스토그램 기록
        embeddings_freq = 1
    )
]

In [35]:
import keras.optimizers

model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(), 
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
)

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 407, 
    epochs=5,
    validation_data = validation_generator, 
    validation_steps=171, 
    callbacks=callbacks_list 
)

2022-02-22 23:27:55.640175: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/5


2022-02-22 23:27:56.284989: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


407/407 [==============================] - ETA: 0s - loss: 2.3484 - binary_accuracy: 0.7567

2022-02-22 23:40:06.983689: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


407/407 [==============================] - 853s 2s/step - loss: 2.3484 - binary_accuracy: 0.7567 - val_loss: 1.6021 - val_binary_accuracy: 0.8198 - lr: 0.0010
Epoch 2/5
407/407 [==============================] - 1075s 3s/step - loss: 2.0282 - binary_accuracy: 0.8117 - val_loss: 1.4346 - val_binary_accuracy: 0.8447 - lr: 0.0010
Epoch 3/5
407/407 [==============================] - 3805s 9s/step - loss: 1.8290 - binary_accuracy: 0.8344 - val_loss: 1.4145 - val_binary_accuracy: 0.8635 - lr: 0.0010
Epoch 4/5
407/407 [==============================] - 841s 2s/step - loss: 1.7733 - binary_accuracy: 0.8424 - val_loss: 2.4424 - val_binary_accuracy: 0.7915 - lr: 0.0010
Epoch 5/5
407/407 [==============================] - 845s 2s/step - loss: 1.7656 - binary_accuracy: 0.8458 - val_loss: 1.4471 - val_binary_accuracy: 0.8666 - lr: 0.0010


In [ ]:
# 미세조정 
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.adam_v2.Adam(learning_rate=1e-5), 
    loss = 'binary_crossentropy',
    metrics = ['binary_accuracy']
)

history = model.fit_generator(
    train_generator,
    steps_per_epoch = 187, 
    epochs=5,
    validation_data = validation_generator, 
    validation_steps=83, 
    callbacks=callbacks_list 
)